In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *
import numpy as np

In [2]:
key_path = key_path
project = project_id

# Sources
table = 'silver_indicators'
dataset = 'silver'
table_conca = f'{project}.{dataset}.{table}'

# Loading tables
table_to_save = 'gold_dim_technical_indicators'
dataset_to_save = 'gold'
table_conca_to_save = f'{project}.{dataset_to_save}.{table_to_save}'


In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [14]:
df = bigquery.run_query(
    f"""
    SELECT
        SMA_20,
        EMA_50,
        WILLR,
        ATR,
        ADX,
        CCI,
        OBV,
        RSI,
        bb_bbh AS BB_BBH,
        daily_return AS DAILY_RETURN,
        macd AS MACD,
        stoch AS STOCH
    FROM {project}.{dataset}.{table} AS indi
    LIMIT 10
    """
)
df

,SMA_20,EMA_50,WILLR,ATR,ADX,CCI,OBV,RSI,BB_BBH,DAILY_RETURN,MACD,STOCH
0,45.063333,46.127450,-8.998563,1.860030,10.549396,57.604726,1.777003e+11,55.555764,558.564140,-0.004628,0.747842,89.985476
1,35.608000,52.389337,-13.821094,2.961766,36.347895,82.380394,1.776783e+11,38.196875,549.132899,-0.020119,-7.511926,47.323232
2,36.503000,47.570096,-1.801812,2.223593,21.442696,117.129447,1.776758e+11,50.256920,550.021969,0.000323,-2.003365,98.919525
3,42.229000,44.026486,-76.292478,2.034497,14.491061,-71.297846,1.776802e+11,43.490675,555.724569,0.050447,-1.201442,15.342018
4,48.966000,47.407311,-8.406796,2.731652,19.428220,101.888047,1.777135e+11,66.736832,562.568102,0.010650,3.170398,93.407195
5,37.971000,55.253707,-10.561960,3.192320,42.358198,95.053320,1.776970e+11,38.822147,551.507376,0.056698,-8.128907,48.222213
6,100.991000,83.444041,-97.320846,5.389773,29.661434,-172.884029,1.777070e+11,48.566391,614.903896,-0.007431,6.631097,52.186800
7,45.444666,45.837059,-95.849061,2.758910,13.641797,-169.525749,1.776637e+11,38.789809,558.960159,-0.178831,-0.420400,36.166252
8,61.064333,58.842854,-2.400823,3.442257,14.111865,151.470232,1.776477e+11,66.997171,574.646745,0.104472,2.911899,86.655546
9,82.548166,73.056836,-2.879629,5.428649,32.121352,149.240617,1.776706e+11,84.418398,597.406338,0.072919,13.977095,97.944596


In [15]:
# Transform the DataFrame to long format (unpivot)
df_unpivot = df.melt(var_name='technical_indicators', value_name='results')

# Drop the 'results' column
df_unpivot = df_unpivot.drop(columns='results')

# Get unique values from the 'technical_indicators' column and convert to DataFrame
unique = pd.DataFrame(df_unpivot['technical_indicators'].unique(), columns=['technical_indicators'])

# Display the result
unique


,technical_indicators
0,SMA_20
1,EMA_50
2,WILLR
3,ATR
4,ADX
5,CCI
6,OBV
7,RSI
8,BB_BBH
9,DAILY_RETURN


In [16]:
bigquery.save_dataframe(unique, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
